In [46]:
!pip install plotly

In [47]:
!pip install dash

In [2]:
import sys
import pandas as pd
import re
import numpy as np
import scipy.io
import pandas as pd

import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

In [3]:
# Batting Data Import
mat = scipy.io.loadmat('matlab.mat')
ary = mat["Data"]
P = np.concatenate(ary["Player"]).ravel()
D = np.concatenate(ary["Date"]).ravel()

AB1 = np.concatenate(ary["AB"]).ravel()
R1 = np.concatenate(ary["R"]).ravel()
H1 = np.concatenate(ary["H"]).ravel()
RBI1 = np.concatenate(ary["RBI"]).ravel()

AB2 = np.concatenate(AB1, axis=0)
R2 = np.concatenate(R1, axis=0)
H2 = np.concatenate(H1, axis=0)
RBI2 = np.concatenate(RBI1, axis=0)


Player = np.concatenate(P)
Date = np.concatenate(D, axis=0)
AB = np.concatenate(AB2, axis=0)
R = np.concatenate(R2, axis=0)
H = np.concatenate(H2, axis=0)
RBI = np.concatenate(RBI2, axis=0)


#df = pd.DataFrame(nArray1, columns=["A", "B", "C", "D", "E", "F"])
dataset = pd.DataFrame({'Date': D, 'Player': Player, 'AB': AB, 'R': R, 'H': H, 'RBI': RBI})
# new data frame with split value columns 
new = dataset["Player"].str.split(",", n = 1, expand = True)
dataset["Player"]= new[0]
dataset["Position"]= new[1]
dataset = dataset[['Date', 'Player', 'Position', 'AB', 'R', 'H', 'RBI']]

In [4]:
## Roster Data Import
roster_file = open('2019combinedroster.txt', 'r')
roster_string = roster_file.read()
roster_df = pd.DataFrame([x.split(',') for x in roster_string.split('\n')])
roster_df['firstinitial'] = roster_df[2].astype(str).str[0]
roster_df["Player"] = roster_df[1] +" "+ roster_df["firstinitial"]
del roster_df[0]
del roster_df[1]
del roster_df[2]
del roster_df[3]
del roster_df[4]
del roster_df[6]
del roster_df['firstinitial']
roster_df = roster_df[["Player",5]]
roster_df.columns = ['Player', 'Team']
roster_df.head()


team_name_file = open('TEAM2019.txt', 'r')
team_name_string = team_name_file.read()
team_name_df = pd.DataFrame([x.split(',') for x in team_name_string.split('\n')])
del team_name_df[1]
team_name_df["teamName"] = team_name_df[2] +" "+ team_name_df[3]
del team_name_df[2]
del team_name_df[3]
team_name_df.columns = ['Team', 'teamName']
team_name_df.head()

#merging datasets
playerTeam = pd.merge(roster_df,  team_name_df,  on ='Team',  how ='left')
dataset = pd.merge(dataset,  playerTeam,  on ='Player',  how ='left')
del dataset["Team"]
dataset['teamName'].astype(str)
dataset

#metric calcs
dataset["AVG"] = dataset["H"]/dataset["AB"]
dataset

,Date,Player,Position,AB,R,H,RBI,teamName,AVG
0,[3/28/2019],Nimmo B,lf,3,0,0,0,New York Mets,0.000000
1,[3/28/2019],Eaton A,rf,3,0,1,0,Washington Nationals,0.333333
2,[3/28/2019],Alonso P,1b,4,0,1,0,New York Mets,0.250000
3,[3/28/2019],Turner T,ss,4,0,2,0,Washington Nationals,0.500000
4,[3/28/2019],Broxton K,pr-rf,0,0,0,0,Baltimore Orioles,NaN
...,...,...,...,...,...,...,...,...,...
2824,[ 9/29/2019],Cimber A,p,0,0,0,0,Cleveland Indians,NaN
2825,[ 9/29/2019],Difo W,3b-ss,4,1,2,0,Washington Nationals,0.500000
2826,[ 9/29/2019],Haase E,ph-c,2,0,0,0,Cleveland Indians,0.000000
2827,[ 9/29/2019],Ross J,p,2,0,0,0,Washington Nationals,0.000000


In [34]:
### App layout

app.layout = html.Div([
    style={'backgroundColor': colors['background']},
    html.H1("MLB Team Management Dashboard", style={'text-align': 'center'}),
    html.Div("Controls", style={'text-align': 'centre'}),
    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2015", "value": 2015},
                     {"label": "2016", "value": 2016},
                     {"label": "2017", "value": 2017},
                     {"label": "2018", "value": 2018}],
                 multi=False,
                 value=2015,
                 style={'width': "40%"}
                 ),
    html.H1("Player", style={'text-align': 'center',
                             'background-color': 'grey',
                             'display':'inline-block', 
                             'width': '50%'}),
    html.H1("Team", style={'text-align': 'center', 
                           'display':'inline-block',
                           'background-color': 'grey',
                           'width': '50%'}),
    
    
    html.Div(id='output_container', children=[]),
    html.Br(),
    
    #dcc.Graph(id='map', figure={}) for output: Output(component_id='map', component_property='figure')

######### TAB stuff ##############
#     app.layout = html.Div([
#     dcc.Tabs(id="tabs", value='tab-1', children=[
#         dcc.Tab(label='Batting', value='tab-1'),
#         dcc.Tab(label='Pitching', value='tab-2'),
#     ]),
#     html.Div(id='tabs-content')
    
# @app.callback(Output('tabs-content', 'children'),
#               [Input('tabs', 'value')])
# def render_content(tab):
#     if tab == 'tab-1':
#         return html.Div([
#             html.H3('Tab content 1')
#         ])
#     elif tab == 'tab-2':
#         return html.Div([
#             html.H3('Tab content 2')
#         ])

])

SyntaxError: invalid syntax (<ipython-input-34-650d278b4d04>, line 52)

In [32]:

# # Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children')],
    [Input(component_id='slct_year', component_property='value')]
)

def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The Year chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["Year"] == option_slctd]
    #dff = dff[dff["Affected by"] == "Varroa_mites"]

     # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )

#     # Plotly Graph Objects (GO)
#     fig = go.Figure(
#         data=[go.Choropleth(
#             locationmode='USA-states',
#             locations=dff['state_code'],
#             z=dff["Pct of Colonies Impacted"].astype(float),
#             colorscale='Reds',
#         )]
#     )
    
#     fig.update_layout(
#         title_text="Bees Affected by Mites in the USA",
#         title_xanchor="center",
#         title_font=dict(size=24),
#         title_x=0.5,
#         geo=dict(scope='usa'),
#     )
#    return container,fig

# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [44]:
colors = {
    'background': '##72DDFA',
    'text': '#FFFFFF'
}
# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")



app.layout = html.Div(style={'backgroundColor': colors['background']},children=[
    html.H1("MLB Team Management Dashboard", style={'text-align': 'center'}),
    html.Div("Controls", style={'text-align': 'centre'}),
    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2015", "value": 2015},
                     {"label": "2016", "value": 2016},
                     {"label": "2017", "value": 2017},
                     {"label": "2018", "value": 2018}],
                 multi=False,
                 value=2015,
                 style={'width': "40%"}
                 ),
    html.H1("Player", style={'text-align': 'center',
                             'background-color': 'grey',
                             'display':'inline-block', 
                             'width': '50%',
                            'height': '50%'}),
    html.H1("Team", style={'text-align': 'center', 
                           'display':'inline-block',
                           'background-color': 'grey',
                           'width': '50%'}),
    
    
    html.Div(id='output_container', children=[]),
    html.Br(),
    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
